In [18]:
import torch

obs_data = torch.rand(8, 3)  # Random tensor of shape [136, 3]
v = obs_data[None, :, :, None].detach()

In [19]:
obs_data

tensor([[0.5260, 0.9231, 0.1846],
        [0.0998, 0.5506, 0.2461],
        [0.8070, 0.2535, 0.6103],
        [0.1831, 0.5146, 0.0623],
        [0.3110, 0.7791, 0.6747],
        [0.4205, 0.0201, 0.9636],
        [0.1119, 0.9773, 0.0910],
        [0.3415, 0.6653, 0.3726]])

In [20]:
v = v.permute(0, 3, 1, 2)
v.shape , v



(torch.Size([1, 1, 8, 3]),
 tensor([[[[0.5260, 0.9231, 0.1846],
           [0.0998, 0.5506, 0.2461],
           [0.8070, 0.2535, 0.6103],
           [0.1831, 0.5146, 0.0623],
           [0.3110, 0.7791, 0.6747],
           [0.4205, 0.0201, 0.9636],
           [0.1119, 0.9773, 0.0910],
           [0.3415, 0.6653, 0.3726]]]]))

In [21]:
n_ped = v.size(-1)
n_ped

3

In [ ]:
temp = v.permute(0, 2, 3, 1).unsqueeze(dim=-2).repeat_interleave(repeats=n_ped, dim=-2)
temp

In [ ]:
temp.shape  ,temp

In [25]:
a = (temp - temp.transpose(2, 3)).norm(p=2, dim=-1)

In [28]:
a

tensor([[[[0.0000, 0.3971, 0.3414],
          [0.3971, 0.0000, 0.7385],
          [0.3414, 0.7385, 0.0000]],

         [[0.0000, 0.4508, 0.1463],
          [0.4508, 0.0000, 0.3045],
          [0.1463, 0.3045, 0.0000]],

         [[0.0000, 0.5535, 0.1967],
          [0.5535, 0.0000, 0.3568],
          [0.1967, 0.3568, 0.0000]],

         [[0.0000, 0.3315, 0.1208],
          [0.3315, 0.0000, 0.4523],
          [0.1208, 0.4523, 0.0000]],

         [[0.0000, 0.4682, 0.3638],
          [0.4682, 0.0000, 0.1044],
          [0.3638, 0.1044, 0.0000]],

         [[0.0000, 0.4004, 0.5431],
          [0.4004, 0.0000, 0.9436],
          [0.5431, 0.9436, 0.0000]],

         [[0.0000, 0.8654, 0.0209],
          [0.8654, 0.0000, 0.8863],
          [0.0209, 0.8863, 0.0000]],

         [[0.0000, 0.3238, 0.0310],
          [0.3238, 0.0000, 0.2927],
          [0.0310, 0.2927, 0.0000]]]])

In [42]:
a_inv = 1. / a

In [43]:
a_inv[a == 0] = 0

In [44]:
a_inv

tensor([[[[ 0.0000,  2.5185,  2.9291],
          [ 2.5185,  0.0000,  1.3542],
          [ 2.9291,  1.3542,  0.0000]],

         [[ 0.0000,  2.2183,  6.8339],
          [ 2.2183,  0.0000,  3.2845],
          [ 6.8339,  3.2845,  0.0000]],

         [[ 0.0000,  1.8067,  5.0830],
          [ 1.8067,  0.0000,  2.8030],
          [ 5.0830,  2.8030,  0.0000]],

         [[ 0.0000,  3.0164,  8.2775],
          [ 3.0164,  0.0000,  2.2108],
          [ 8.2775,  2.2108,  0.0000]],

         [[ 0.0000,  2.1360,  2.7490],
          [ 2.1360,  0.0000,  9.5789],
          [ 2.7490,  9.5789,  0.0000]],

         [[ 0.0000,  2.4973,  1.8411],
          [ 2.4973,  0.0000,  1.0598],
          [ 1.8411,  1.0598,  0.0000]],

         [[ 0.0000,  1.1555, 47.8959],
          [ 1.1555,  0.0000,  1.1283],
          [47.8959,  1.1283,  0.0000]],

         [[ 0.0000,  3.0887, 32.2144],
          [ 3.0887,  0.0000,  3.4163],
          [32.2144,  3.4163,  0.0000]]]])

In [46]:
a_hat = a_inv + torch.eye(n=n_ped, device=v.device)

In [48]:
node_degrees = a_hat.sum(dim=-1).unsqueeze(dim=-1)
degs_inv_sqrt = torch.pow(node_degrees, -0.5)
degs_inv_sqrt[torch.isinf(degs_inv_sqrt)] = 0
norm_degs_matrix = torch.eye(n=n_ped, device=v.device) * degs_inv_sqrt

In [49]:
a = torch.eye(n=n_ped, device=v.device) - norm_degs_matrix @ a_hat @ norm_degs_matrix

In [52]:
a = a.squeeze(dim=0).detach()

In [53]:
a.shape

torch.Size([8, 3, 3])